In [1]:
from __future__ import annotations
from pathlib import Path
import hashlib
import re
import pandas as pd

In [2]:
def find_fase1_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if candidate.name == 'fase_1_diagnostico':
            return candidate
    raise RuntimeError('Nao encontrei fase_1_diagnostico a partir deste notebook')

CURRENT_DIR = Path.cwd().resolve()
FASE1_ROOT = find_fase1_root(CURRENT_DIR)
SOURCE_PATH = FASE1_ROOT / 'dados' / 'manifestos_final' / 'manifest_arquivos_classificados.csv'
OUTPUT_CSV = FASE1_ROOT / 'dados' / 'manifestos_final' / 'manifest_unificado.csv'
OUTPUT_PARQUET = OUTPUT_CSV.with_suffix('.parquet')

FASE1_ROOT, SOURCE_PATH.exists(), OUTPUT_CSV.parent.exists()

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico'), True, True)

In [3]:
df = pd.read_csv(SOURCE_PATH)
len(df), df.columns.tolist()[:5]

(145,
 ['path_local', 'pasta_raiz', 'nome_arquivo', 'extensao', 'tamanho_bytes'])

In [4]:
def safe_value(value, default: str) -> str:
    if pd.isna(value):
        return default
    value_str = str(value).strip()
    return value_str if value_str else default

def build_id(path_local: str, hash_sha: str) -> str:
    raw = f"{path_local}::{hash_sha}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()

YEAR_PATTERN = re.compile(r"(19|20)\\d{2}")
MONTH_PATTERN = re.compile(r"(?<!\\d)(0[1-9]|1[0-2])(?!\\d)")

def guess_year_month(nome_arquivo: str | None) -> tuple[str, str]:
    nome = (nome_arquivo or "").lower()
    year_match = YEAR_PATTERN.search(nome)
    year = year_match.group(0) if year_match else "0000"
    month_match = MONTH_PATTERN.search(nome)
    month = month_match.group(1) if month_match else "00"
    return year, month

def build_path_minio(row: pd.Series) -> tuple[str, list[str]]:
    year, month = guess_year_month(row.get("nome_arquivo"))
    dominio = safe_value(row.get("dominio"), "desconhecido")
    forno = safe_value(row.get("forno"), "nao_definido")
    granularidade = safe_value(row.get("granularidade"), "nao_definida")
    nome_arquivo = safe_value(row.get("nome_arquivo"), "arquivo_sem_nome")
    path = f"{dominio}/{forno}/{granularidade}/{year}/{month}/{nome_arquivo}"
    notes: list[str] = []
    if year == "0000" or month == "00":
        notes.append("path_minio com ano/mes placeholder")
    if granularidade == "nao_definida":
        notes.append("granularidade nao_definida")
    if forno == "nao_definido":
        notes.append("forno nao_definido")
    if dominio == "desconhecido":
        notes.append("dominio desconhecido")
    return path, notes

In [5]:
df['id_arquivo'] = df.apply(lambda row: build_id(row['path_local'], row['hash_sha256']), axis=1)
df['origem_fisica'] = 'local'
df['bucket_minio'] = 'maringa-raw'
df['status_pipeline'] = 'raw_local'
if 'fonte_manifesto' not in df.columns:
    if 'fontes_manifesto' in df.columns:
        df['fonte_manifesto'] = df['fontes_manifesto']
    else:
        df['fonte_manifesto'] = 'consolidado_2_3'
path_info = df.apply(build_path_minio, axis=1, result_type='expand')
df['path_minio'] = path_info[0]
df['observacoes_pipeline'] = path_info[1].apply(lambda notes: '; '.join(notes) if notes else None)
df[['id_arquivo', 'path_minio', 'observacoes_pipeline']].head(3)

,id_arquivo,path_minio,observacoes_pipeline
0,a6f218c760553643e00509da74840389201504f8a73214...,consumo_fornos/F2/nao_definida/0000/01/2018_F2...,path_minio com ano/mes placeholder; granularid...
1,89eefd15fa128e850286d69f879cb7e22a499778b46575...,consumo_fornos/F3/nao_definida/0000/01/2018_F3...,path_minio com ano/mes placeholder; granularid...
2,180edc6cb677c648c10930d3121aeb8495546f673a146e...,consumo_fornos/F4/nao_definida/0000/01/2018_F4...,path_minio com ano/mes placeholder; granularid...


In [6]:
priority_cols = [
    'id_arquivo',
    'path_local',
    'pasta_raiz',
    'pasta_raiz_canonica',
    'nome_arquivo',
    'extensao',
    'tamanho_bytes',
    'data_modificacao',
    'hash_sha256',
    'dominio',
    'forno',
    'granularidade',
    'origem_fisica',
    'bucket_minio',
    'path_minio',
    'status_pipeline',
    'fonte_manifesto',
    'observacoes_dominio',
    'observacoes_forno',
    'observacoes_granularidade',
    'observacoes_pipeline'
]
remaining_cols = [col for col in df.columns if col not in priority_cols]
ordered_cols = priority_cols + remaining_cols
sort_cols = ['dominio', 'forno', 'pasta_raiz_canonica', 'nome_arquivo']
df_ordered = df.sort_values(sort_cols).reset_index(drop=True)[ordered_cols]
df_ordered.head(3)

,id_arquivo,path_local,pasta_raiz,pasta_raiz_canonica,nome_arquivo,extensao,tamanho_bytes,data_modificacao,hash_sha256,dominio,...,path_minio,status_pipeline,fonte_manifesto,observacoes_dominio,observacoes_forno,observacoes_granularidade,observacoes_pipeline,fontes_manifesto,qtd_arquivos_reportada_manifesto,descricao_manifesto
0,0000b88858fe4a0371b224eb49a4f47f544c709322f38e...,dados/dados_iniciais/Consumo Fornos/2019_F1_Co...,Consumo Fornos,Consumo Fornos,2019_F1_Consumo.csv,.csv,42566405,2025-04-29T15:23:46+00:00,e2d6122f72312561518cd3dfbcfc425ca210fa1140a2e1...,consumo_fornos,...,consumo_fornos/F1/nao_definida/0000/01/2019_F1...,raw_local,"index_resumo_por_pasta,manifest_index",NaN,NaN,Sem indicacao clara para consumo_fornos,path_minio com ano/mes placeholder; granularid...,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."
1,e684247a48fd1fbc45f967124314b326cbc886b004fac0...,dados/dados_iniciais/Consumo Fornos/2020_F1_Co...,Consumo Fornos,Consumo Fornos,2020_F1_Consumo.csv,.csv,28785370,2025-04-29T15:25:02+00:00,e2f659eb79b9efb6d1a1aedf577b2f5ee9ce0b38cc8309...,consumo_fornos,...,consumo_fornos/F1/nao_definida/0000/02/2020_F1...,raw_local,"index_resumo_por_pasta,manifest_index",NaN,NaN,Sem indicacao clara para consumo_fornos,path_minio com ano/mes placeholder; granularid...,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."
2,d2243389167f2f277659ebe70c82160fefd9ad15680e62...,dados/dados_iniciais/Consumo Fornos/2021_F1_Co...,Consumo Fornos,Consumo Fornos,2021_F1_Consumo.csv,.csv,29619879,2025-04-29T15:26:22+00:00,bca0695b76a186d6e603ddc6775d2379d4d094d2f66f89...,consumo_fornos,...,consumo_fornos/F1/nao_definida/0000/02/2021_F1...,raw_local,"index_resumo_por_pasta,manifest_index",NaN,NaN,Sem indicacao clara para consumo_fornos,path_minio com ano/mes placeholder; granularid...,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."


In [7]:
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_ordered.to_csv(OUTPUT_CSV, index=False)
parquet_written = False
parquet_error = None
try:
    df_ordered.to_parquet(OUTPUT_PARQUET, index=False)
    parquet_written = True
except Exception as exc:
    parquet_error = str(exc)
OUTPUT_CSV, parquet_written, parquet_error

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico/dados/manifestos_final/manifest_unificado.csv'),
 False,
 "Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.\nA suitable version of pyarrow or fastparquet is required for parquet support.\nTrying to import the above resulted in these errors:\n - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.\n - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.")

In [8]:
resumo_dominio = (
    df_ordered
    .groupby('dominio', dropna=False)
    .agg(qtd_arquivos=('path_local', 'size'), tamanho_total_bytes=('tamanho_bytes', 'sum'))
    .sort_values('qtd_arquivos', ascending=False)
    .reset_index()
)
resumo_dominio

,dominio,qtd_arquivos,tamanho_total_bytes
0,corridas,40,25655908
1,informacoes_diarias,40,3439738
2,consumo_fornos,39,1132715472
3,dicionario,21,1345169
4,supervisorio,4,963334195
5,eletrodo,1,20604


In [9]:
resumo_forno = (
    df_ordered
    .groupby('forno', dropna=False)
    .agg(qtd_arquivos=('path_local', 'size'), tamanho_total_bytes=('tamanho_bytes', 'sum'))
    .sort_values('qtd_arquivos', ascending=False)
    .reset_index()
)
resumo_forno

,forno,qtd_arquivos,tamanho_total_bytes
0,F5,32,280077728
1,F4,29,1163049803
2,F2,24,230411262
3,F3,24,235878545
4,F1,23,216566941
5,nao_definido,13,526807


In [10]:
resumo_status = (
    df_ordered
    .groupby('status_pipeline', dropna=False)
    .agg(qtd_arquivos=('path_local', 'size'), tamanho_total_bytes=('tamanho_bytes', 'sum'))
    .reset_index()
)
resumo_status

,status_pipeline,qtd_arquivos,tamanho_total_bytes
0,raw_local,145,2126511086


In [11]:
samples_por_pasta = (
    df_ordered
    .groupby('pasta_raiz_canonica', group_keys=False)
    .head(2)
    .loc[:, [
        'pasta_raiz_canonica',
        'nome_arquivo',
        'dominio',
        'forno',
        'granularidade',
        'id_arquivo',
        'bucket_minio',
        'path_minio',
        'status_pipeline',
        'observacoes_pipeline'
    ]]
 )
samples_por_pasta

,pasta_raiz_canonica,nome_arquivo,dominio,forno,granularidade,id_arquivo,bucket_minio,path_minio,status_pipeline,observacoes_pipeline
0,Consumo Fornos,2019_F1_Consumo.csv,consumo_fornos,F1,nao_definida,0000b88858fe4a0371b224eb49a4f47f544c709322f38e...,maringa-raw,consumo_fornos/F1/nao_definida/0000/01/2019_F1...,raw_local,path_minio com ano/mes placeholder; granularid...
1,Consumo Fornos,2020_F1_Consumo.csv,consumo_fornos,F1,nao_definida,e684247a48fd1fbc45f967124314b326cbc886b004fac0...,maringa-raw,consumo_fornos/F1/nao_definida/0000/02/2020_F1...,raw_local,path_minio com ano/mes placeholder; granularid...
39,Corridas,2018_F1_Corrida.csv,corridas,F1,corrida,6de33c9c4895f9026fecf31fa2ca95ca7f1008279ee45e...,maringa-raw,corridas/F1/corrida/0000/01/2018_F1_Corrida.csv,raw_local,path_minio com ano/mes placeholder
40,Corridas,2019_F1_Corrida.csv,corridas,F1,corrida,0efb28f79ff32e878144fae19ceef1a8e3cf8852fb29ba...,maringa-raw,corridas/F1/corrida/0000/01/2019_F1_Corrida.csv,raw_local,path_minio com ano/mes placeholder
79,Dicionario,Dic_Supervisorio F4.pdf,dicionario,F4,meta,10ff58281d67d61679701b0aaf0b5696c611450498f0f0...,maringa-raw,dicionario/F4/meta/0000/00/Dic_Supervisorio F4...,raw_local,path_minio com ano/mes placeholder
80,Dicionario,Dic_Supervisorio F4.txt,dicionario,F4,meta,a4c0b185d213016aa0c300b523243ea95899a56c405595...,maringa-raw,dicionario/F4/meta/0000/00/Dic_Supervisorio F4...,raw_local,path_minio com ano/mes placeholder
100,Eletrodo,Medição Eletrodo.csv,eletrodo,nao_definido,nao_definida,ccdac107338076d38320d04c0edc786c407701f692bce8...,maringa-raw,eletrodo/nao_definido/nao_definida/0000/00/Med...,raw_local,path_minio com ano/mes placeholder; granularid...
101,Informacoes Diaria,2018_F1_Inf.Diario.csv,informacoes_diarias,F1,dia,7af6e5979718d634dde8ca20098f03eabfa71e149e8bd8...,maringa-raw,informacoes_diarias/F1/dia/0000/01/2018_F1_Inf...,raw_local,path_minio com ano/mes placeholder
102,Informacoes Diaria,2019_F1_Inf.Diario.csv,informacoes_diarias,F1,dia,d20c9abaaf4207088d9899287e8246f1bd7d1ea592885e...,maringa-raw,informacoes_diarias/F1/dia/0000/01/2019_F1_Inf...,raw_local,path_minio com ano/mes placeholder
141,Supervisorio Forno 4,F4_2024_1S.csv,supervisorio,F4,minuto,fd8ca8a7994f88b7347dddbc7ee942bd39ab8689126b07...,maringa-raw,supervisorio/F4/minuto/0000/02/F4_2024_1S.csv,raw_local,path_minio com ano/mes placeholder
